Test

In [1]:
import os
import random
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
distance = 15
energy = 600
doubleplane = 30
neutron = 4
erel = 500

In [3]:
vmcworkdir = os.environ["VMCWORKDIR"]
os.environ["GEOMPATH"] = vmcworkdir + "/geometry"
os.environ["CONFIG_DIR"] = vmcworkdir + "/gconfig"
os.environ["PHYSICSLIST"] = "QGSP_INCLXX_HP"

In [4]:
run = ROOT.FairRunSim()
run.SetName("TGeant4")
run.SetStoreTraj(False)
run.SetMaterials("media_r3b.geo")

run.SetSink(ROOT.FairRootFileSink("test.root"))

generator = ROOT.FairPrimaryGenerator()
inputfile = "%s/input/%dSn_%dn_%dAMeV_%dkeV.dat" % (
    vmcworkdir,
    132 - neutron,
    neutron,
    energy,
    erel,
)
generator.AddGenerator(ROOT.R3BAsciiGenerator(inputfile))
run.SetGenerator(generator)

cave = ROOT.R3BCave("Cave")
cave.SetGeometryFileName("r3b_cave_vacuum.geo")
run.AddModule(cave)

neuland_position = ROOT.TGeoTranslation(0.0, 0.0, distance + doubleplane * 10.0 / 2.0)
neuland = ROOT.R3BNeuland(doubleplane, neuland_position)
run.AddModule(neuland)

magnetic_field = ROOT.R3BGladFieldMap("R3BGladMap")
magnetic_field.SetScale(-0.6)
run.SetField(magnetic_field)

run.Init()
ROOT.TVirtualMC.GetMC().SetRandom(ROOT.TRandom3(random.randint(0, 10000)))
ROOT.TVirtualMC.GetMC().SetMaxNStep(100000)

rtdb = run.GetRuntimeDb()
parout = ROOT.FairParRootFileIo(True)
parout.open("test.para.root")
rtdb.setOutput(parout)
rtdb.saveOutput()

run.Run(10)

[INFO] Media file used: /data/jmayer/Simulations/R3BRoot/geometry/media_r3b.geo
[INFO] R3BAsciiGenerator: Looking for ions...
[INFO] R3BAsciiGenerator: Reading input file /data/jmayer/Simulations/R3BRoot/input/128Sn_4n_600AMeV_500keV.dat
[INFO] New Ion Ion_128_50 Z=50  A=128  Charge=50  Mass=127.911 GeV 
[INFO] R3BAsciiGenerator: 1 ions registered.
[INFO] FairRootFileSink initialized.
[INFO]  - cbmroot
[INFO]     - test.root
[INFO] ==============  FairRunSim: Initialising simulation run ==============
[INFO] FairGeoMedia: Read media 

************************************************************* 
     initialisation for run id 1584016657
************************************************************* 
-I- R3BGladFieldMap: Reading field map from ASCII file /data/jmayer/Simulations/R3BRoot/field/magField/R3B/R3BGladMap.dat
-I- R3BGladFieldMap: 401841 entries to read...   0   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  2

Error in <TProtoClass::FindDataMember>: data member with index 0 is not found in class locale
Error in <CreateRealData>: Cannot find data member # 0 of class locale for parent R3BAsciiGenerator!
Error in <TProtoClass::FindDataMember>: data member with index 0 is not found in class locale
Error in <CreateRealData>: Cannot find data member # 0 of class locale for parent R3BAsciiGenerator!
Info in <TGeoManager::TGeoManager>: Geometry FAIRGeom, FAIR geometry created
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::CheckGeometry>: Fixing runtime shapes...
Info in <TGeoManager::CheckGeometry>: ...Nothing to fix
Info in <TGeoManager::CloseGeometry>: Counting nodes...
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CloseGeometry>: Building cache...
Info in <TGeoManager::CountLevels>: max level = 3, max placements = 3000
Info in <TGeoManager::CloseG

In [5]:
os.remove("test.root")
os.remove("test.para.root")